# K2DSE Benchmarks

This is a lab report of currents experiments with KDSE*

## What do we need to show 

### Goal 0 - re-implementation


First, I need to make sure my re-implementation is identical. Not trivial as we count SD differently.

#### 0.1 Count that the number of SD found is the same between OldKDSE,DKDSE,DKDSEA (the capstone implementation) and KDSE,K2DSE,K2DSEA (the paper re-implementation).

&#10004;  We are correct

#### 0.2 Verify that the algorithm isnt slower

&#10004;  We are faster 

#### 0.3 Check that the thread implementation is identical and also faster.

&#10004;  The implementation can explore more point given the concurrency effect over the end of exploration.
On large instances, there is a clear speed-up (when available).

### Goal 1 - Execution Time and size of explored space by DSE methods. 

#### generate the table 1 with KDSE,K2DSE,K2DSEA (no multi-thread).


### Goal 2 - Pareto fronts and explored space 

#### generate the Fig 3 with K2DSE,K2DSEA,PDSE (no multi-thread).

## Prepare data

In [ ]:
import dsereader

logdir = "../kdse2023_log/"


applications = {
    "bipartite" :  { "name" : "bipartite" },
   "Echo" :  { "name" : "Echo" },
    "fig8" :  { "name" : "fig8" },
   "H264" :  { "name" : "H264" },
    "modem" :  { "name" : "modem" },
    "sample" :  { "name" : "sample" },
    "satellite" :  { "name" : "satellite" },
    "BlackScholes" :  { "name" : "BlackScholes" },
    "example" :  { "name" : "example" },
   "h263decoder" :  { "name" : "h263decoder" },
   "JPEG2000" :  { "name" : "JPEG2000" },
    "PDectect" :  { "name" : "PDectect" },
    "samplerate"  :  { "name" : "samplerate" }
}

methods = {
 #  0 Infos
 #  1 Throughput   
 #   2 : { "name" : "OldKDSE" ,   "color" : "black"}, # "-aKPeriodicThroughputwithDSE"
 #   3 : { "name" : "DeepKDSE" ,  "color" : "red"}, # "-aDeepKPeriodicThroughputwithDSE"
 #   4 : { "name" : "DeepKDSEA" , "color" : "green"}, # "-aDeepKPeriodicThroughputwithDSE -papprox=1"
 #   5 : { "name" : "KDSE" ,      "color" : "black"}, # "-athroughputbufferingDSE -prealtime=1 -pmode=KDSE"
    2 : { "name" : "K2DSE" ,     "color" : "black"}, # "-athroughputbufferingDSE -prealtime=1 -pmode=K2DSE"
    3 : { "name" : "K2DSEA" ,     "color" : "black"}, # "-athroughputbufferingDSE -prealtime=1 -pmode=K2DSEA"
#    8 : { "name" : "KDSE2" ,     "color" : "black"}, # "-athroughputbufferingDSE -prealtime=1 -pmode=KDSE -pthread=2"
#    8 : { "name" : "KDSE4" ,     "color" : "black"}, # "-athroughputbufferingDSE -prealtime=1 -pmode=KDSE -pthread=4"
#   10 : { "name" : "KDSE8" ,     "color" : "black"}, # "-athroughputbufferingDSE -prealtime=1 -pmode=KDSE -pthread=8"
#    9 : { "name" : "KDSE16" ,    "color" : "black"}, # "-athroughputbufferingDSE -prealtime=1 -pmode=KDSE -pthread=16"
}

In [ ]:
## Collect the max throughput for each application
import pandas as pd 

for app in applications.keys() :
    for line in open(logdir + "/" + app + "_1.txt").read().split("\n"):
        if 'KPeriodic Throughput is' in line :
            th = float(line.split(" ")[-1])
            applications[app]["max_throughput"] = th
    for line in open(logdir + "/" + app + "_0.txt").read().split("\n"):
        if 'Task count' in line :
            count = int(line.split(" ")[-1])
            applications[app]["task_count"] = count

In [ ]:
dsereader.plot_all(logdir, graphs=applications.keys(), methods=methods, plotfunc=dsereader.plot_app_pareto)

In [ ]:
import math
import datetime
def time_in_msec(time_msec): # copy pasted from https://stackoverflow.com/questions/48063828/convert-duration-format-from-float-to-monthdayshoursminutesseconds-in-python
    time_sec = int(time_msec // 1000)
    delta = datetime.timedelta(seconds=time_sec)
    delta_str = str(delta)[-8:]
    hours, minutes, seconds = [int(val) for val in delta_str.split(":", 3)]
    weeks = delta.days // 7
    days = delta.days % 7
    return "{}days {}h {}min {}.{}sec ({})".format(days, hours, minutes, seconds,int(time_msec) & 1000, time_msec)

time_in_msec(100000.10)


In [ ]:
def gen_dse_data(logdir, applications, methods, columns = ["throughput", 
                                                           "storage distribution size", 
                                                           "cumulative duration"]):
    list_of_dict = []

    for key,values in applications.items():
        
        app = key
        app_name = values["name"]
        app_max_throughput = values["max_throughput"]
        app_task_count = values["task_count"]
        for m in methods:
            method_name = methods[m]["name"]
            try :
                df = dsereader.load_app_dse(logdir, app, m, cols = columns)
            except FileNotFoundError:
                df = pd.Dataframe()
            sd_count = df["storage distribution size"].count() if "throughput" in df else "-"
            max_th = df["throughput"].max() 
            duration = df["cumulative duration"].max()  if "cumulative duration" in df else "-"
            print(app, m, max_th, app_max_throughput)
            finished = math.isclose(max_th, app_max_throughput, rel_tol=1e-5)
            pareto = dsereader.extract_pareto(df[["throughput","storage distribution size"]])
            pareto_count = pareto["storage distribution size"].count() if finished else "-"
            list_of_dict += [{"graph" : app_name, 
                                "#task" : app_task_count,
                                "method" : method_name, 
                                "#SD" : sd_count,
                                "#Pareto" : pareto_count,
                                "Duration" : duration, 
                                "Finished" : finished}]
            
    return pd.DataFrame(list_of_dict)

In [ ]:
df = gen_dse_data(logdir, applications=applications, methods=methods)
df.set_index(["graph","#task","method"])


In [ ]:
colformat = "|".join([""] + ["l"] * df.index.nlevels + ["r"] * df.shape[1] + [""])
               
latex = df.to_latex(
        float_format="{:0.1f}".format # , column_format=colformat, index=False
    )

# Check new implementation

## 0.1 Check the correctness of the new algorithm

❌ We explore less for fig8,and 
❌ We explore more for BlackScholes. This is due to a difference in OldKDSE when they initialize the first SD. they are correct with this particular app, but their init could be wrong. We stick to the curren one, can be improved later.
❌ There are strange artefact when looking at sample output from OldKDSE, it sets buffers to values higher than required at initilization, it is a bug in OldKDSE.


In [ ]:
methods_to_compare = ["OldKDSE", "KDSE"]

df = gen_dse_data(logdir, applications=applications, methods=methods)
subdf = df[df["method"].isin(methods_to_compare)]
subdf

In [ ]:
methods_to_compare = ["DeepKDSE", "K2DSE"]

df = gen_dse_data(logdir, applications=applications, methods=methods)
subdf = df[df["method"].isin(methods_to_compare)]
subdf

In [ ]:
methods_to_compare = ["DeepKDSEA", "K2DSEA"]

df = gen_dse_data(logdir, applications=applications, methods=methods)
subdf = df[df["method"].isin(methods_to_compare)]
subdf

## 0.2 and 0.3 Check non-threaded and threaded versions are faster

When effective we gain one order of magnitude.
When not, we lose a few seconds maximum.
On my machine 16 is too much.

In [ ]:
import seaborn as sns
sns.set_style('whitegrid')

df = gen_dse_data(logdir, applications=applications, methods=methods)

ax=sns.lineplot(data=df, x="method", y="Duration", hue="graph", marker='o', markersize=5)
_ = ax.set(yscale='log')

sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))


## 0.3bis Check threaded version has no duplicates

The following test ensure the threaded version does not explore twice the same point. 

In [ ]:
def sanity_check(logdir, applications, methods):
    for app_key,app_values in applications.items():
        for method_key,method_values in methods.items():
            
            app_name = app_values["name"]
            app_task_count = app_values["task_count"]
            method_name = method_values["name"]
            
            try :
                df = dsereader.load_app_dse(logdir, app_key, method_key, cols = ["throughput", 
                                                                                 "storage distribution size",
                                                                                 "cumulative duration",
                                                                                 "feedback quantities"])
            except FileNotFoundError:
                continue
            except ValueError:
                continue

            # assert it finished
            max_th = df["throughput"].max() 
            app_max_throughput = app_values["max_throughput"]
            finished = math.isclose(max_th, app_max_throughput, rel_tol=1e-5)
            assert(finished)

            # assert there is no duplicates
            duplicates_count = len(df["feedback quantities"]) - len(df["feedback quantities"].drop_duplicates())
            assert(duplicates_count == 0)

            print(app_key, method_key, finished, duplicates_count)
        
    return True

In [ ]:
sanity_check(logdir, applications=applications, methods=methods)

# What do we do next

The KDSE2 algorithm is recursive. The first level is global, the second level is local. 

## Improve the local level

There is no way to get deeper, the local level is a single cycle. But single cycle are usually small, so we can accelerate this part by using static knowledge. For example the distribution size for a cycle must be greater than X to improve the throughput. 

## Improve the global level

At the global level, instead of restarting the local level again and again for a same cycle, we could use a cache that store the result.